In [1]:
import numpy as np
import pandas as pd
import os
import librosa
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [2]:
from sklearn.metrics import accuracy_score
from scipy.stats import skew
SAMPLE_RATE = 16000

In [3]:
from tqdm import tqdm

import scipy

In [43]:
librosa.show_versions()

INSTALLED VERSIONS
------------------
python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]

librosa: 0.10.1

audioread: 3.0.1
numpy: 1.26.0
scipy: 1.11.3
sklearn: 1.3.2
joblib: 1.3.2
decorator: 5.1.1
numba: 0.58.1
soundfile: 0.9.0
pooch: v1.8.0
soxr: 0.3.7
typing_extensions: installed, no version number available
lazy_loader: installed, no version number available
msgpack: 1.0.7

numpydoc: None
sphinx: None
sphinx_rtd_theme: None
matplotlib: 3.8.1
sphinx_multiversion: None
sphinx_gallery: None
mir_eval: None
ipython: None
sphinxcontrib.rsvgconverter: None
pytest: None
pytest_mpl: None
pytest_cov: None
samplerate: None
resampy: None
presets: None
packaging: 23.1


In [4]:
df = pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/data/train.csv")

In [5]:
def get_mfcc(name):
    path = "/scratch/network/mk8574/audio_sentiment_challenge/data"
    b, _ = librosa.core.load(path + name[1:], sr = SAMPLE_RATE)

   
    assert _ == SAMPLE_RATE
 
    try:
      
        ft1 = librosa.feature.mfcc(y=b, sr = SAMPLE_RATE, n_mfcc=20)

        ft2 = librosa.feature.zero_crossing_rate(y=b)[0]
        ft3 = librosa.feature.spectral_rolloff(y=b)[0]
        ft4 = librosa.feature.spectral_centroid(y=b)[0]
        
        ft1_trunc = np.hstack((np.mean(ft1, axis=1), np.std(ft1, axis=1), skew(ft1, axis = 1), np.max(ft1, axis = 1), np.min(ft1, axis = 1)))
        ft2_trunc = np.hstack((np.mean(ft2), np.std(ft2), skew(ft2), np.max(ft2), np.min(ft2)))
        ft3_trunc = np.hstack((np.mean(ft3), np.std(ft3), skew(ft3), np.max(ft3), np.min(ft3)))
        ft4_trunc = np.hstack((np.mean(ft4), np.std(ft4), skew(ft4), np.max(ft4), np.min(ft4)))
        
        return pd.Series(np.hstack((ft1_trunc, ft2_trunc, ft3_trunc, ft4_trunc)))
    except:
        print('bad file')
        return pd.Series([0]*115)

In [6]:
get_mfcc(df["path"][0])

0      -414.755768
1       110.100639
2        46.699074
3        23.939814
4        14.766221
          ...     
110    1757.048961
111     548.601215
112       4.651649
113    4937.061334
114    1417.299571
Length: 115, dtype: float64

In [7]:
train = pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/data/train.csv")
test = pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/data/test.csv")
train_data = pd.DataFrame()
train_data['fname'] = train['path']
test_data = pd.DataFrame()
test_data['fname'] = test['path']

train_data = train_data['fname'].apply(get_mfcc)
print('done loading train mfcc')
test_data = test_data['fname'].apply(get_mfcc)
print('done loading test mfcc')

train_data['fname'] = train['path']
test_data['fname'] = test['path']

/tmp/ipykernel_514581/3758001970.py:16: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  ft1_trunc = np.hstack((np.mean(ft1, axis=1), np.std(ft1, axis=1), skew(ft1, axis = 1), np.max(ft1, axis = 1), np.min(ft1, axis = 1)))


done loading train mfcc
done loading test mfcc


In [8]:
train_data['label'] = train['label']
test_data['label'] = np.zeros((len(test)))

In [9]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,107,108,109,110,111,112,113,114,fname,label
0,-414.755768,110.100639,46.699074,23.939814,14.766221,4.820827,1.445079,-0.926153,2.892507,4.465991,...,2.716211,9700.708008,2433.251953,1757.048961,548.601215,4.651649,4937.061334,1417.299571,./train/TRAIN_0000.wav,1
1,-399.769531,83.051300,55.473316,31.782587,22.040754,0.985082,3.712758,-2.359320,-0.026562,0.393657,...,0.739956,10012.939453,2583.984375,2684.697354,1668.877531,1.220943,6634.829822,1377.601065,./train/TRAIN_0001.wav,2
2,-341.145081,97.399071,38.274349,19.811539,0.731027,0.838703,-1.911201,-10.645285,-1.595292,-2.305276,...,0.810742,9851.440430,1313.525391,2378.619653,1515.726845,1.998935,6632.407966,1255.889075,./train/TRAIN_0002.wav,4
3,-376.963715,118.961670,34.490349,24.178419,-1.065605,-1.613391,2.888372,-7.865878,-2.260844,-5.609925,...,1.045671,8074.951172,1012.060547,1557.806416,510.669550,2.322108,4122.733019,1010.040479,./train/TRAIN_0003.wav,5
4,-352.863251,117.553345,29.948687,31.094316,5.406392,-5.591999,-4.809400,-11.500416,-6.894948,2.946273,...,0.052639,7644.287109,1248.925781,1673.094057,446.903984,1.873157,3786.299328,975.654634,./train/TRAIN_0004.wav,4


In [55]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,107,108,109,110,111,112,113,114,fname,label
0,-335.757324,125.215431,22.145767,14.351713,-1.045250,0.567860,-0.666743,-8.114830,-7.136740,-0.890951,...,0.181159,6610.693359,1378.125000,1579.666105,304.585754,0.392215,2542.704594,1027.372377,./test/TEST_0000.wav,0.0
1,-295.973083,92.839691,24.976181,22.831312,-10.278670,7.804742,-7.779604,-11.835020,-9.290084,-2.127427,...,0.543844,9130.078125,1152.026367,1954.635065,504.731338,1.578950,4243.908179,1038.011646,./test/TEST_0001.wav,0.0
2,-444.395996,109.385201,55.236778,26.486050,12.487712,9.069915,2.790433,-0.485669,-2.200089,-3.334618,...,1.030299,10034.472656,872.094727,1692.604231,893.836349,3.206180,6174.399779,971.531068,./test/TEST_0002.wav,0.0
3,-384.600281,112.463974,47.454556,17.336460,13.556947,4.677102,-5.572524,-1.354830,-2.386852,0.361085,...,1.068405,10002.172852,872.094727,1878.747055,1117.230823,3.416088,7019.040575,888.376030,./test/TEST_0003.wav,0.0
4,-273.304077,97.817047,12.370096,24.591480,3.248469,-9.987857,-6.246260,-13.113686,-1.204894,0.282557,...,0.390368,8031.884766,1862.622070,2104.891983,518.462117,1.418307,4013.059597,1384.532166,./test/TEST_0004.wav,0.0


In [10]:
def extract_features(files):
    path = "/scratch/network/mk8574/audio_sentiment_challenge/data"
    features = {}
    cnt = 0
    for f in tqdm(files):
        features[f] = {}
        fs, data = scipy.io.wavfile.read(path + f[1:])
        abs_data = np.abs(data)
        diff_data = np.diff(data)
        def calc_part_features(data, n=2, prefix=''):
            f_i = 1
            for i in range(0, len(data), len(data)//n):
                features[f]['{}mean_{}_{}'.format(prefix, f_i, n)] = np.mean(data[i:i + len(data)//n])
                features[f]['{}std_{}_{}'.format(prefix, f_i, n)] = np.std(data[i:i + len(data)//n])
                features[f]['{}min_{}_{}'.format(prefix, f_i, n)] = np.min(data[i:i + len(data)//n])
                features[f]['{}max_{}_{}'.format(prefix, f_i, n)] = np.max(data[i:i + len(data)//n])
        features[f]['len'] = len(data)
        if features[f]['len'] > 0:
            n = 1
            calc_part_features(data, n=n)
            calc_part_features(abs_data, n=n, prefix='abs_')
            calc_part_features(diff_data, n=n, prefix='diff_')

            n = 2
            calc_part_features(data, n=n)
            calc_part_features(abs_data, n=n, prefix='abs_')
            calc_part_features(diff_data, n=n, prefix='diff_')

            n = 3
            calc_part_features(data, n=n)
            calc_part_features(abs_data, n=n, prefix='abs_')
            calc_part_features(diff_data, n=n, prefix='diff_')
        cnt += 1
    features = pd.DataFrame(features).T.reset_index()
    features.rename(columns={'index': 'fname'}, inplace=True)
    return features


train_files = train.path.values
train_features = extract_features(train_files)


test_files = test.path.values
test_features = extract_features(test_files)

100%|██████████| 1881/1881 [00:02<00:00, 639.53it/s]


In [11]:
train_features

,fname,len,mean_1_1,std_1_1,min_1_1,max_1_1,abs_mean_1_1,abs_std_1_1,abs_min_1_1,abs_max_1_1,...,min_1_3,max_1_3,abs_mean_1_3,abs_std_1_3,abs_min_1_3,abs_max_1_3,diff_mean_1_3,diff_std_1_3,diff_min_1_3,diff_max_1_3
0,./train/TRAIN_0000.wav,37904.0,-0.355134,223.039507,-1364.0,1370.0,169.978023,144.409901,0.0,1370.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
1,./train/TRAIN_0001.wav,41642.0,-0.359733,259.496480,-1517.0,1149.0,197.045387,168.853985,0.0,1517.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
2,./train/TRAIN_0002.wav,46446.0,-1.012509,910.592771,-6442.0,7420.0,498.087263,762.292135,0.0,7420.0,...,-4570.0,5337.0,333.710632,524.232158,0.0,5337.0,0.000000,0.000000,0.0,0.0
3,./train/TRAIN_0003.wav,36837.0,-0.654749,959.786176,-7604.0,7286.0,520.772457,806.217080,0.0,7604.0,...,-934.0,891.0,133.694112,114.320855,0.0,934.0,0.000000,0.000000,0.0,0.0
4,./train/TRAIN_0004.wav,29897.0,-0.272435,1326.059113,-14531.0,17987.0,590.387832,1187.381595,0.0,17987.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,./train/TRAIN_4996.wav,45379.0,-0.229379,403.370622,-2608.0,3398.0,248.264462,317.919280,0.0,3398.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.008925,38.115913,-902.0,1023.0
4997,./train/TRAIN_4997.wav,40574.0,1.826662,5252.274526,-32768.0,32767.0,2310.475304,4716.788621,-32768.0,32767.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
4998,./train/TRAIN_4998.wav,35235.0,-1.941876,627.238480,-4727.0,5149.0,370.769746,505.926553,0.0,5149.0,...,-4727.0,5149.0,429.149425,685.473437,0.0,5149.0,-0.500000,0.500000,-1.0,0.0
4999,./train/TRAIN_4999.wav,50717.0,-0.510815,287.863414,-1662.0,2302.0,202.494568,204.600968,0.0,2302.0,...,-13.0,-11.0,12.000000,1.000000,11.0,13.0,2.000000,0.000000,2.0,2.0


In [12]:
train_data = train_data.merge(train_features, on='fname', how='left')
test_data = test_data.merge(test_features, on='fname', how='left')
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,min_1_3,max_1_3,abs_mean_1_3,abs_std_1_3,abs_min_1_3,abs_max_1_3,diff_mean_1_3,diff_std_1_3,diff_min_1_3,diff_max_1_3
0,-414.755768,110.100639,46.699074,23.939814,14.766221,4.820827,1.445079,-0.926153,2.892507,4.465991,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,-399.769531,83.051300,55.473316,31.782587,22.040754,0.985082,3.712758,-2.359320,-0.026562,0.393657,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,-341.145081,97.399071,38.274349,19.811539,0.731027,0.838703,-1.911201,-10.645285,-1.595292,-2.305276,...,-4570.0,5337.0,333.710632,524.232158,0.0,5337.0,0.0,0.0,0.0,0.0
3,-376.963715,118.961670,34.490349,24.178419,-1.065605,-1.613391,2.888372,-7.865878,-2.260844,-5.609925,...,-934.0,891.0,133.694112,114.320855,0.0,934.0,0.0,0.0,0.0,0.0
4,-352.863251,117.553345,29.948687,31.094316,5.406392,-5.591999,-4.809400,-11.500416,-6.894948,2.946273,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
c = pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/prob_plus.csv")

train_data  =  pd.concat([train_data,c],axis=1)

In [14]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,prob_2,prob_3,prob_4,prob_5,prob_0.1,prob_1.1,prob_2.1,prob_3.1,prob_4.1,prob_5.1
0,-414.755768,110.100639,46.699074,23.939814,14.766221,4.820827,1.445079,-0.926153,2.892507,4.465991,...,9.972004e-13,7.481026e-10,2.284406e-11,1.174704e-05,0.114615,0.004971,0.000283,0.007948,0.000094,0.872090
1,-399.769531,83.051300,55.473316,31.782587,22.040754,0.985082,3.712758,-2.359320,-0.026562,0.393657,...,5.319619e-14,2.680158e-10,3.765693e-11,9.999982e-01,0.001258,0.001267,0.000135,0.000204,0.000346,0.996790
2,-341.145081,97.399071,38.274349,19.811539,0.731027,0.838703,-1.911201,-10.645285,-1.595292,-2.305276,...,1.147402e-09,3.107179e-10,1.000000e+00,1.520057e-09,0.006031,0.003878,0.012162,0.002638,0.973926,0.001364
3,-376.963715,118.961670,34.490349,24.178419,-1.065605,-1.613391,2.888372,-7.865878,-2.260844,-5.609925,...,3.311203e-13,1.407155e-10,9.452709e-01,5.423207e-02,0.035601,0.001977,0.003281,0.001916,0.500885,0.456340
4,-352.863251,117.553345,29.948687,31.094316,5.406392,-5.591999,-4.809400,-11.500416,-6.894948,2.946273,...,9.764557e-01,1.408232e-03,1.648795e-09,1.984731e-10,0.001558,0.466801,0.295990,0.234605,0.000560,0.000486


In [62]:
X = test_data.drop(['label', 'fname'], axis=1)
print(list(test_data.columns))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 'fname', 'label', 'len', 'mean_1_1', 'std_1_1', 'min_1_1', 'max_1_1', 'abs_mean_1_1', 'abs_std_1_1', 'abs_min_1_1', 'abs_max_1_1', 'diff_mean_1_1', 'diff_std_1_1', 'diff_min_1_1', 'diff_max_1_1', 'mean_1_2', 'std_1_2', 'min_1_2', 'max_1_2', 'abs_mean_1_2', 'abs_std_1_2', 'abs_min_1_2', 'abs_max_1_2', 'diff_mean_1_2', 'diff_std_1_2', 'diff_min_1_2', 'diff_max_1_2', 'mean_1_3', 'std_1_3', 'min_1_3', 'max_1_3', 'abs_mean_1_3', 'abs_std_1_3', 'abs_min_1_3', 'abs_max_1_3', 'diff_mean_1_3', 'diff_std_1_3', 'diff_min_1_3', 'diff_max_1_

In [15]:
x = train_data.drop(columns = ["fname","label"])
y = train_data["label"]
x.head()

,0,1,2,3,4,5,6,7,8,9,...,prob_2,prob_3,prob_4,prob_5,prob_0.1,prob_1.1,prob_2.1,prob_3.1,prob_4.1,prob_5.1
0,-414.755768,110.100639,46.699074,23.939814,14.766221,4.820827,1.445079,-0.926153,2.892507,4.465991,...,9.972004e-13,7.481026e-10,2.284406e-11,1.174704e-05,0.114615,0.004971,0.000283,0.007948,0.000094,0.872090
1,-399.769531,83.051300,55.473316,31.782587,22.040754,0.985082,3.712758,-2.359320,-0.026562,0.393657,...,5.319619e-14,2.680158e-10,3.765693e-11,9.999982e-01,0.001258,0.001267,0.000135,0.000204,0.000346,0.996790
2,-341.145081,97.399071,38.274349,19.811539,0.731027,0.838703,-1.911201,-10.645285,-1.595292,-2.305276,...,1.147402e-09,3.107179e-10,1.000000e+00,1.520057e-09,0.006031,0.003878,0.012162,0.002638,0.973926,0.001364
3,-376.963715,118.961670,34.490349,24.178419,-1.065605,-1.613391,2.888372,-7.865878,-2.260844,-5.609925,...,3.311203e-13,1.407155e-10,9.452709e-01,5.423207e-02,0.035601,0.001977,0.003281,0.001916,0.500885,0.456340
4,-352.863251,117.553345,29.948687,31.094316,5.406392,-5.591999,-4.809400,-11.500416,-6.894948,2.946273,...,9.764557e-01,1.408232e-03,1.648795e-09,1.984731e-10,0.001558,0.466801,0.295990,0.234605,0.000560,0.000486


In [78]:
import sklearn
scaler = sklearn.preprocessing.MinMaxScaler()
x.columns = x.columns.astype(str)
np_scaled = scaler.fit_transform(x)

print(x.columns)
x = pd.DataFrame(np_scaled, columns=x.columns)

x.head()

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       'min_1_3', 'max_1_3', 'abs_mean_1_3', 'abs_std_1_3', 'abs_min_1_3',
       'abs_max_1_3', 'diff_mean_1_3', 'diff_std_1_3', 'diff_min_1_3',
       'diff_max_1_3'],
      dtype='object', length=152)


,0,1,2,3,4,5,6,7,8,9,...,min_1_3,max_1_3,abs_mean_1_3,abs_std_1_3,abs_min_1_3,abs_max_1_3,diff_mean_1_3,diff_std_1_3,diff_min_1_3,diff_max_1_3
0,0.710759,0.787311,0.802330,0.591960,0.788204,0.692588,0.598161,0.732436,0.699421,0.557772,...,0.998689,0.004527,0.000000,0.000000,0.995473,0.000000,0.438926,0.0,0.988411,0.003757
1,0.725623,0.593886,0.906042,0.733270,0.909050,0.601665,0.664935,0.698480,0.595883,0.459934,...,0.998689,0.004527,0.000000,0.000000,0.995473,0.000000,0.438926,0.0,0.988411,0.003757
2,0.783768,0.696484,0.702749,0.517578,0.555048,0.598196,0.499333,0.502158,0.540241,0.395092,...,0.859407,0.166667,0.036156,0.054123,0.995473,0.162877,0.438926,0.0,0.988411,0.003757
3,0.748242,0.850675,0.658022,0.596259,0.525202,0.540071,0.640660,0.568012,0.516634,0.315698,...,0.970223,0.031596,0.014485,0.011803,0.995473,0.028504,0.438926,0.0,0.988411,0.003757
4,0.772146,0.840604,0.604339,0.720869,0.632716,0.445762,0.413994,0.481898,0.352265,0.521261,...,0.998689,0.004527,0.000000,0.000000,0.995473,0.000000,0.438926,0.0,0.988411,0.003757


In [16]:
X_train , X_test , y_train, y_test = train_test_split(x,y , test_size=0.1, random_state=42)

In [17]:
xgb = XGBClassifier(max_depth=5, learning_rate=0.05, n_estimators=5000,
                    n_jobs=-1, random_state=0, reg_alpha=0.2, 
                    colsample_bylevel=0.9, colsample_bytree=0.9) #1000개의 가지? epoch? , 0.05 학습률
xgb.fit(X_train, y_train) #학습

y_preds = xgb.predict(X_test) #검증

print('Accuracy: %.2f' % accuracy_score(y_test,y_preds))

Accuracy: 0.52


In [2]:
import pandas as pd

In [10]:
wprob = pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/prob_whisper.csv")

In [11]:
prob = pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/prob.csv")

In [12]:
over= []
down=[]
for i in range(len(prob)):
    if prob["prob_"+str(prob["label"][i])][i]>0.7:
        over.append(i)
    else:
        down.append(i)
print(len(over),len(down))

1577 304


In [13]:
over= []
down=[]
for i in range(len(wprob)):
    if prob["prob_"+str(wprob["label"][i])][i]>0.7:
        over.append(i)
    else:
        down.append(i)
print(len(over),len(down))

1282 599


In [113]:
testing= test_data.drop(columns = ["fname","label"])

test_preds = xgb.predict(testing)
test_proba = xgb.predict_proba(testing)

In [38]:
import numpy as np
new_prob = pd.DataFrame()
new_prob['id'] = prob['id']
new_prob['label'] = np.zeros((len(prob)))
for i in range(6):
    new_prob["prob_"+str(i)]= np.zeros((len(prob)))
for i in range(len(prob)):
    for j in range(6):
        new_prob["prob_"+str(j)][i]=wprob["prob_"+str(j)][i]*0.7+prob["prob_"+str(j)][i]

new_prob.head()

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



,id,label,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5
0,TEST_0000,0.0,0.001956,0.000509,0.000432,0.000272,0.005476,1.691356
1,TEST_0001,0.0,0.730369,0.761059,0.007696,0.054015,0.001754,0.145107
2,TEST_0002,0.0,0.000258,0.014070,1.669594,0.001468,0.014426,0.000184
3,TEST_0003,0.0,0.003342,0.000479,0.000328,0.700809,0.000856,0.994186
4,TEST_0004,0.0,1.670401,0.000405,0.000859,0.004143,0.023067,0.001125


In [18]:
new_prob.tail()

,id,label,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5
1876,TEST_1876,0.0,1.949614,0.007440,0.000265,0.002929,0.000166,0.039586
1877,TEST_1877,0.0,0.007854,0.000932,0.003245,1.986007,0.000147,0.001815
1878,TEST_1878,0.0,0.525888,0.000353,0.000810,1.466418,0.000183,0.006348
1879,TEST_1879,0.0,0.000459,0.000357,0.003866,0.311396,1.683738,0.000184
1880,TEST_1880,0.0,0.213174,0.140008,0.094193,0.041022,1.484256,0.027347


In [ ]:
new = pd.DataFrame()
new['id'] = prob['id']
new['label'] = np.zeros((len(prob)))
for i in range(len(prob)):
    a=0
    k = -1
    for j in range(6):
        if new_prob["prob_"+str(j)][i]>a:
            a = new_prob["prob_"+str(j)][i]
            k=j
    new["label"][i]=int(k)

In [40]:
new= new.astype({'label':'int'})
new.head()


,id,label
0,TEST_0000,5
1,TEST_0001,1
2,TEST_0002,2
3,TEST_0003,5
4,TEST_0004,0


In [ ]:
242 1639

In [44]:
a,b = 0,0
for i in range(len(prob)):
    if prob["label"][i]!=new["label"][i]:
        a+=1
    else:
        b+=1
print(a,b)

205 1676


In [45]:
a,b = 0,0
for i in range(len(prob)):
    if wprob["label"][i]!=new["label"][i]:
        a+=1
    else:
        b+=1
print(a,b)

260 1621


In [46]:
a,b = 0,0
for i in range(len(prob)):
    if wprob["label"][i]!=prob["label"][i]:
        a+=1
    else:
        b+=1
print(a,b)

464 1417


In [26]:
new_sub =new.drop(['prob_0','prob_1','prob_2','prob_3','prob_4','prob_5'], axis=1)

KeyError: "['prob_0', 'prob_1', 'prob_2', 'prob_3', 'prob_4', 'prob_5'] not found in axis"

In [25]:
print(new_sub)

             id  label
0     TEST_0000    0.0
1     TEST_0001    0.0
2     TEST_0002    0.0
3     TEST_0003    0.0
4     TEST_0004    0.0
...         ...    ...
1876  TEST_1876    0.0
1877  TEST_1877    0.0
1878  TEST_1878    0.0
1879  TEST_1879    0.0
1880  TEST_1880    0.0

[1881 rows x 2 columns]


In [43]:
new.to_csv("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/new2.csv", index=False)

In [114]:
test_proba

array([[3.8718931e-02, 6.7146681e-02, 2.8629117e-03, 9.3227647e-02,
        1.1289854e-01, 6.8514532e-01],
       [2.7429497e-01, 4.2957246e-01, 1.7026683e-03, 2.8915199e-02,
        2.3358066e-04, 2.6528105e-01],
       [7.7873183e-04, 7.9524098e-03, 4.4832677e-01, 4.8667914e-01,
        5.5464290e-02, 7.9872360e-04],
       ...,
       [1.4940101e-01, 3.7658857e-03, 1.0225133e-03, 1.2609671e-01,
        1.8676501e-04, 7.1952713e-01],
       [4.1025676e-04, 1.1387312e-01, 3.7755233e-01, 9.8026894e-02,
        1.4306578e-01, 2.6707160e-01],
       [1.6689297e-02, 5.1863849e-02, 4.2232943e-01, 1.7720972e-01,
        2.6968962e-01, 6.2218044e-02]], dtype=float32)

In [105]:
print(test_preds)

[5 1 3 ... 5 2 2]


In [32]:
a,b=0,0
for i in range(len(prob)):
    if prob["label"][i]==new["label"][i]:
        a+=1
    else:
        b+=1
print(a,b,a/(a+b))

1501 380 0.797979797979798


In [33]:
a,b=0,0
for i in range(len(prob)):
    if wprob["label"][i]==new["label"][i]:
        a+=1
    else:
        b+=1
print(a,b,a/(a+b))

1795 86 0.9542796384901648


In [37]:
a,b=0,0
for i in down:
    if new["label"][i]==prob["label"][i]:
        a+=1
    else:
        b+=1
print(a,b,a/(a+b))

219 380 0.3656093489148581


In [112]:
a,b=0,0
for i in over:
    if prob["label"][i]==test_preds[i]:
        a+=1
    else:
        b+=1
print(a,b,a/(a+b))

902 639 0.5853341985723556


In [ ]:
for i in range(len(p))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test,y_preds)

plt.figure(figsize=(16,9))
sns.heatmap(
    cm,
    annot=True,
    xticklabels=["blues","classical","country","disco","hiphop","jazz","metal","pop","reggae","rock"],
    yticklabels=["blues","classical","country","disco","hiphop","jazz","metal","pop","reggae","rock"]
)
plt.show()

In [52]:
import random
import os
import numpy as np
import torch
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [75]:
seed_everything(42)

In [81]:
from sklearn.model_selection import train_test_split
train = pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/data/train.csv")
print(len(train))
a,b = train_test_split(train,test_size =0.2,random_state = 42)

5001


In [80]:
print(len(train))

NameError: name 'train' is not defined

In [82]:
from sklearn.model_selection import train_test_split
train = pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/data/train.csv")
x = train["path"]
y = train["label"]
print(len(train))
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size =0.2,random_state = 42)

5001


In [86]:
x_train

4677    ./train/TRAIN_4677.wav
800     ./train/TRAIN_0800.wav
3671    ./train/TRAIN_3671.wav
4193    ./train/TRAIN_4193.wav
2968    ./train/TRAIN_2968.wav
                 ...          
4426    ./train/TRAIN_4426.wav
466     ./train/TRAIN_0466.wav
3092    ./train/TRAIN_3092.wav
3772    ./train/TRAIN_3772.wav
860     ./train/TRAIN_0860.wav
Name: path, Length: 4000, dtype: object

In [87]:
train = pd.read_csv("/scratch/network/mk8574/audio_sentiment_challenge/data/train.csv")

In [89]:
tr,tst =  train_test_split(train,test_size =0.2,random_state = 42)

In [90]:
tr

,id,path,label
4677,TRAIN_4677,./train/TRAIN_4677.wav,0
800,TRAIN_0800,./train/TRAIN_0800.wav,3
3671,TRAIN_3671,./train/TRAIN_3671.wav,2
4193,TRAIN_4193,./train/TRAIN_4193.wav,2
2968,TRAIN_2968,./train/TRAIN_2968.wav,1
...,...,...,...
4426,TRAIN_4426,./train/TRAIN_4426.wav,0
466,TRAIN_0466,./train/TRAIN_0466.wav,3
3092,TRAIN_3092,./train/TRAIN_3092.wav,2
3772,TRAIN_3772,./train/TRAIN_3772.wav,2


In [91]:
tst

,id,path,label
1501,TRAIN_1501,./train/TRAIN_1501.wav,5
2586,TRAIN_2586,./train/TRAIN_2586.wav,5
2653,TRAIN_2653,./train/TRAIN_2653.wav,4
1055,TRAIN_1055,./train/TRAIN_1055.wav,4
705,TRAIN_0705,./train/TRAIN_0705.wav,2
...,...,...,...
2313,TRAIN_2313,./train/TRAIN_2313.wav,4
3214,TRAIN_3214,./train/TRAIN_3214.wav,5
2732,TRAIN_2732,./train/TRAIN_2732.wav,2
1926,TRAIN_1926,./train/TRAIN_1926.wav,2


In [92]:
val =tst.drop("label",axis=1)

In [93]:
val

,id,path
1501,TRAIN_1501,./train/TRAIN_1501.wav
2586,TRAIN_2586,./train/TRAIN_2586.wav
2653,TRAIN_2653,./train/TRAIN_2653.wav
1055,TRAIN_1055,./train/TRAIN_1055.wav
705,TRAIN_0705,./train/TRAIN_0705.wav
...,...,...
2313,TRAIN_2313,./train/TRAIN_2313.wav
3214,TRAIN_3214,./train/TRAIN_3214.wav
2732,TRAIN_2732,./train/TRAIN_2732.wav
1926,TRAIN_1926,./train/TRAIN_1926.wav


In [94]:
val.to_csv("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/valid.csv", index=False)

In [95]:
tst.to_csv("/scratch/network/mk8574/audio_sentiment_challenge/baseline_dy/xgboost/validans.csv", index=False)

In [ ]:


valid3 = valid1.merge(valid2, how='left')
